In [1]:
import cobra
import pandas as pd
import scipy
import docplex
import cplex
from src.compy import compy
import os
import sys

src_path = os.path.abspath(os.path.join(os.getcwd(), 'src'))

# Add to sys.path if not already there
if src_path not in sys.path:
    sys.path.insert(0, src_path)

from src.diet_adaptation import *
from tqdm import tqdm
from optlang import Model as OptModel, Variable, Constraint, Objective
from concurrent.futures import ProcessPoolExecutor
import pandas as pd

In [2]:
compy(abun_filepath="test_data_input/normCoverageReduced.csv",
     mod_filepath='test_data_input/AGORA103',
     out_filepath="Python_Models2/Personalized",
     diet_filepath='test_data_input/AverageEU_diet_fluxes.txt',
     workers=2)

----------Starting ComPy pipeline----------
----------Reading abundance file----------
**********Building global community model**********
**********Added model: Bacteroides_cellulosilyticus_DSM_14838**********


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e
c:\Users\agarg\AppData\Local\Programs\Python\Python311\Lib\site-packages\cobra\core\model.py:779: UserWarning: need to pass in a list
  warn("need to pass in a list")
c:\Users\agarg\AppData\Local\Programs\Python\Python311\Lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")
No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e


**********Added model: Parabacteroides_merdae_ATCC_43184**********


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e


**********Added model: Bacteroides_vulgatus_ATCC_8482**********


...

No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e


**********Added model: Bacteroides_ovatus_ATCC_8483**********


No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e


**********Added model: Bacteroides_sp_2_1_33B**********
----------Finished adding GEM reconstructions to community----------
----------Adding diet and fecal compartments----------
----------Done adding diet and fecal compartments----------


Building sample models: 100%|██████████| 4/4 [01:59<00:00, 29.91s/it]


In [9]:
from src.diet_adaptation import *
clean_samp_names, organisms, ex_mets = get_individual_size_name(
    abun_file_path='test_data_input/normCoverageReduced.csv',
    mod_path='test_data_input/AGORA103'
)

No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e
No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e
No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e
No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e
No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e
No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression 

In [11]:
exchanges, net_production, net_uptake = simulate_microbiota_models(
    samp_names=clean_samp_names,
    ex_mets=ex_mets,
    model_dir='Python_Models2/Personalized',
    diet_file='test_data_input/AverageEU_diet_fluxes.txt',
    res_path='Python_Models2',
    lower_bm=0.4,
    upper_bm=1.0,
    solver='cplex',
    workers=2
)

c:\Users\agarg\research\Microbiome-GEM-Modeling\src\diet_adaptation.py:165: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-187.5472124059008' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  diet_df.at[index, 'upper_bound'] = -0.8 * row['original_lb']


Got constraints, starting parallel processing


Processing samples: 100%|██████████| 4/4 [07:58<00:00, 119.70s/it]


All samples processed successfully


In [12]:
net_secretion_df, net_uptake_df = collect_flux_profiles(
    samp_names=clean_samp_names,
    exchanges=sorted(exchanges),
    net_production=net_production,
    net_uptake=net_uptake
)
net_secretion_df

,SRS011061,SRS012902,SRS011134,SRS054956
EX_12dgr180[fe],0.000000,0.000000,0.000000,0.000000
EX_12ppd_S[fe],0.000000,0.000000,0.000000,0.000000
EX_26dap_M[fe],106.692178,109.973211,20.073655,0.075309
EX_2hyoxplac[fe],0.000925,0.000032,0.003248,0.000113
EX_2obut[fe],218.511066,136.851051,61.402039,55.406226
...,...,...,...,...
EX_xan[fe],0.000000,0.000000,0.000000,0.000000
EX_xyl_D[fe],0.000000,0.000000,0.000000,0.000000
EX_xylan[fe],0.000000,0.000000,0.000000,0.000000
EX_xyluglc[fe],0.000000,0.000000,0.000000,0.000000


In [ ]:
compy(abun_filepath="test_data_input/normCoverageReduced.csv",
     mod_filepath='test_data_input/AGORA103',
     out_filepath="Python_Models2/Personalized",
     diet_filepath='test_data_input/AverageEU_diet_fluxes.txt',
     workers=2)

clean_samp_names, organisms, ex_mets = get_individual_size_name(
    abun_file_path='test_data_input/normCoverageReduced.csv',
    mod_path='test_data_input/AGORA103'
)

exchanges, net_production, net_uptake = simulate_microbiota_models(
    samp_names=clean_samp_names,
    ex_mets=ex_mets,
    model_dir='Python_Models2/Personalized',
    diet_file='test_data_input/AverageEU_diet_fluxes.txt',
    res_path='Python_Models2',
    lower_bm=0.4,
    upper_bm=1.0,
    solver='cplex',
    workers=2
)

net_secretion_df, net_uptake_df = collect_flux_profiles(
    samp_names=clean_samp_names,
    exchanges=sorted(exchanges),
    net_production=net_production,
    net_uptake=net_uptake
)

net_secretion_df.to_csv('python_net_secretion_fluxes.csv', index=True, index_label='Net secretion')
net_uptake_df.to_csv('python_net_uptake_fluxes.csv', index=True, index_label='Net uptake')

c:\Users\agarg\research\Microbiome-GEM-Modeling\src\diet_adaptation.py:165: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-187.5472124059008' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  diet_df.at[index, 'upper_bound'] = -0.8 * row['original_lb']


Got constraints, starting parallel processing


Processing samples: 100%|██████████| 4/4 [06:13<00:00, 93.28s/it] 


All samples processed successfully


## Make Python model equivalent of Matlab Feasible

In [ ]:
clean_samp_names, organisms, ex_mets = get_individual_size_name(
    abun_file_path='test_data_input/normCoverageReduced.csv',
    mod_path='test_data_input/AGORA103'
)

In [ ]:
os.makedirs('Python_Models2\Diet', exist_ok=True)
exchanges = [f"EX_{m.replace('[e]', '[fe]')}" for m in ex_mets if m != 'biomass[e]']

net_production = {}
net_uptake = {}

In [ ]:
# define human-derived metabolites present in the gut: primary bile acids, amines, mucins, host glycans
humanMets = {
    'gchola': -10, 'tdchola': -10, 'tchola': -10, 'dgchol': -10,
    '34dhphe': -10, '5htrp': -10, 'Lkynr': -10, 'f1a': -1,
    'gncore1': -1, 'gncore2': -1, 'dsT_antigen': -1, 'sTn_antigen': -1,
    'core8': -1, 'core7': -1, 'core5': -1, 'core4': -1,
    'ha': -1, 'cspg_a': -1, 'cspg_b': -1, 'cspg_c': -1,
    'cspg_d': -1, 'cspg_e': -1, 'hspg': -1
}

# Adapt diet
diet_constraints = adapt_vmh_diet_to_agora('test_data_input/AverageEU_diet_fluxes.txt', setup_used='Microbiota')

In [ ]:
model = load_matlab_model('zzzresults\microbiota_model_samp_SRS011061.mat')
model_data = loadmat('zzzresults\microbiota_model_samp_SRS011061.mat', simplify_cells=True)['model']
model.solver = 'cplex'
model.name = 'SRS011061'

In [ ]:
diet_rxns = [r.id for r in model.reactions if '[d]' in r.id and r.id.startswith('EX_')]
for rxn_id in diet_rxns:
    new_id = rxn_id.replace('EX_', 'Diet_EX_')
    if new_id not in model.reactions:
        model.reactions.get_by_id(rxn_id).id = new_id

# First: Set ALL Diet_EX_ reactions to lower bound 0 (like useDiet does)
for rxn in model.reactions:
    if rxn.id.startswith('Diet_EX_'):
        rxn.lower_bound = 0

# Apply diet
for _, row in diet_constraints.iterrows():
    rxn = row['rxn_id']
    if rxn in model.reactions:
        model.reactions.get_by_id(rxn).lower_bound = float(row['lower_bound'])
        if pd.notnull(row['upper_bound']):
            model.reactions.get_by_id(rxn).upper_bound = float(row['upper_bound'])

print(f"Processing 'SRS011061': diet applied")

In [ ]:
if 'communityBiomass' in model.reactions:
    model.reactions.communityBiomass.lower_bound = 0.4
    model.reactions.communityBiomass.upper_bound = 1.0

for rxn in model.reactions:
    if rxn.id.startswith('UFEt_') or rxn.id.startswith('DUt_') or rxn.id.startswith('EX_'):
        rxn.upper_bound = 1e6

# Change the bound of the humanMets if not included in the diet
# BUT it is in the existing model's reactions
for met_id, bound in humanMets.items():
    rxn_id = f'Diet_EX_{met_id}[d]'
    if rxn_id not in diet_constraints['rxn_id'].values and rxn_id in model.reactions:
        model.reactions.get_by_id(rxn_id).bounds = bound, 10000.


# close demand and limit sink reactions
for rxn in model.reactions:
    if '_DM_' in rxn.id:
        rxn.lower_bound = 0
    elif '_sink_' in rxn.id:
        rxn.lower_bound = -1 

In [ ]:
for rxn in model.reactions:
    if 'Diet_EX_adocbl[d]' in rxn.id:
        print(rxn.id, rxn.bounds)

In [ ]:
model.objective = 'EX_microbeBiomass[fe]'
model.optimize()

In [ ]:
# Save the diet-adapted model
save_dir = os.path.join('zzzresults', 'Diet')
os.makedirs(save_dir, exist_ok=True)

model_dict = make_mg_pipe_model_dict(
    model, C=model_data['C'], d=model_data['d'], dsense=model_data['dsense'], ctrs=model_data['ctrs']
)

diet_model_path = os.path.join(save_dir, f"microbiota_model_diet_SRS011061.mat")
savemat(diet_model_path, {'model': model_dict}, oned_as='column')

In [ ]:
fecal_rxn_ids = [model.reactions.index(rxn) for rxn in model.exchanges]

diet_rxn_ids = [rxn.id.replace('EX_', 'Diet_EX_').replace('[fe]', '[d]') for rxn in model.exchanges]
diet_rxn_ids = [model.reactions.index(model.reactions.get_by_id(rid)) for rid in diet_rxn_ids if rid in model.reactions]

A, rhs, csense, lb, ub, c = build_constraint_matrix(diet_model_path)

In [ ]:
opt_model, vars, obj_expr = build_optlang_model(A, rhs, csense, lb, ub, c)

In [ ]:
opt_model.objective = Objective(obj_expr, direction='max')
print(f'Model Status after optimization: {opt_model.optimize()}')

In [ ]:
matmodel = cobra.io.load_matlab_model('Matlab_Models/Diet/microbiota_model_diet_SRS011061.mat')
mfecal_rxn_ids = [matmodel.reactions.index(rxn) for rxn in matmodel.exchanges]

mdiet_rxn_ids = [rxn.id.replace('EX_', 'Diet_EX_').replace('[fe]', '[d]') for rxn in matmodel.exchanges]
mdiet_rxn_ids = [matmodel.reactions.index(matmodel.reactions.get_by_id(rid)) for rid in diet_rxn_ids if rid in matmodel.reactions]

mA, mrhs, mcsense, mlb, mub, mc = build_constraint_matrix('Matlab_Models/Diet/microbiota_model_diet_SRS011061.mat')

mopt_model, mvars, mobj_expr = build_optlang_model(mA, mrhs, mcsense, mlb, mub, mc)

mopt_model.objective = Objective(mobj_expr, direction='max')
print(f'Model Status after optimization: {mopt_model.optimize()}')

In [ ]:
import pandas as pd

def normalize_expression(expr):
    expr = '+' + expr
    expr = expr.replace('- ', '-').replace('+- ', '-').replace('+-', '-').replace('+ ', '+')

    def round_match(match):
        num = float(match.group())
        return f"{round(num, 4)}"

    expr = re.sub(r'-?\d+\.\d+', round_match, expr)
    
    expr_list = expr.split(' ')
    return ' '.join(sorted(expr_list)) 

def build_expr_map(constraints, source):
    expr_map = {}
    for s in constraints:
        expr = s.split(', ')[1]
        lb = s.split(', ')[2]
        ub = s.split(', ')[3]
        norm_expr = normalize_expression(expr)
        expr_map[norm_expr] = {'source': source, 'lb': lb, 'ub': ub}
    return expr_map

py_map = build_expr_map(pyconstraints, "only_python")
mat_map = build_expr_map(matconstraints, "only_matlab")

# Merge both with preference for "both" if found in both
records = []

all_exprs = set(py_map.keys()).union(set(mat_map.keys()))

for expr in all_exprs:
    if expr in py_map and expr in mat_map:
        records.append({
            'expression': expr,
            'source': 'both',
            'lb': (py_map[expr]['lb'], mat_map[expr]['lb']),
            'ub': (py_map[expr]['ub'], mat_map[expr]['ub'])
        })
    elif expr in py_map:
        records.append({
            'expression': expr,
            'source': 'only_python',
            'lb': py_map[expr]['lb'],
            'ub': py_map[expr]['ub']
        })
    else:
        records.append({
            'expression': expr,
            'source': 'only_matlab',
            'lb': mat_map[expr]['lb'],
            'ub': mat_map[expr]['ub']
        })

common = len(set(py_map.keys()).intersection(set(mat_map.keys())))
py = len(set(py_map.keys()) - set(mat_map.keys()))
mat = len(set(py_map.keys()) - set(mat_map.keys()))

print(common, py, mat)
# Create the DataFrame
df = pd.DataFrame(records)
df

In [ ]:
import pandas as pd

def get_value_differences(dict1, dict2):
    """
    Compares two dictionaries with the same keys and returns a new dictionary
    containing only the keys where the values differ, along with their values
    from both original dictionaries.
    """
    diff_dict = {}
    for key in dict1.keys():  # Assuming dict1 and dict2 have the same keys
        if dict1[key] != dict2[key]:
            diff_dict[key] = {'dict1_value': dict1[key], 'dict2_value': dict2[key]}
    return diff_dict

differences = get_value_differences(vars1, vars2)

# Build row-by-row records from the differences
diff_rows = []
for rxn_id, val in differences.items():
    lb1, ub1, flux1 = val['dict1_value']
    lb2, ub2, flux2 = val['dict2_value']
    diff_rows.append({
        "Reaction ID": rxn_id,
        "Model1_LB": lb1,
        "Model1_UB": ub1,
        "Model1_Flux": flux1,
        "Model2_LB": lb2,
        "Model2_UB": ub2,
        "Model2_Flux": flux2,
        "Flux_Diff": abs(flux1 - flux2)
    })

# Create and sort DataFrame
df_diffs = pd.DataFrame(diff_rows)
df_diffs = df_diffs.sort_values(by="Flux_Diff", ascending=False)
df_diffs

In [ ]:
mopt_model.objective.value, opt_model.objective.value

In [ ]:
for idx, rxn in enumerate(model.reactions):
    if rxn.id == 'EX_microbeBiomass[fe]': print(idx, rxn.id, rxn.bounds)

from cobra.flux_analysis import flux_variability_analysis
fva = flux_variability_analysis(model, reaction_list=["EX_microbeBiomass[fe]"], fraction_of_optimum=1.0)
print(fva)

## Copy below but with Python Personalized Model

In [ ]:

personalized_path = 'Python_Models2\Personalized\pymicrobiota_model_samp_SRS011061.mat'

model = cobra.io.load_matlab_model(personalized_path)

samp_names = ['SRS011061']
model_dir = 'test_data_input/AGORA103'
diet_file = 'test_data_input\AverageEU_diet_fluxes.txt'
res_path = 'zzzresults/Matlab'
lower_bm=0.4 
upper_bm=1.0
solver='cplex'

os.makedirs(res_path, exist_ok=True)
exchanges = [f"EX_{m.replace('[e]', '[fe]')}" for m in ex_mets if m != 'biomass[e]']

net_production = {}
net_uptake = {}

# define human-derived metabolites present in the gut: primary bile acids, amines, mucins, host glycans
humanMets = {
    'gchola': -10, 'tdchola': -10, 'tchola': -10, 'dgchol': -10,
    '34dhphe': -10, '5htrp': -10, 'Lkynr': -10, 'f1a': -1,
    'gncore1': -1, 'gncore2': -1, 'dsT_antigen': -1, 'sTn_antigen': -1,
    'core8': -1, 'core7': -1, 'core5': -1, 'core4': -1,
    'ha': -1, 'cspg_a': -1, 'cspg_b': -1, 'cspg_c': -1,
    'cspg_d': -1, 'cspg_e': -1, 'hspg': -1
}

# Adapt diet
diet_constraints = adapt_vmh_diet_to_agora(diet_file, setup_used='Microbiota')

for samp in tqdm(samp_names):
    model = model
    model_data = loadmat(personalized_path, simplify_cells=True)['model']
    model.solver = solver
    model.name = samp

    print(f"Processing {samp}: got model")

    # Before applying diet constraints, update reaction IDs in the model
    diet_rxns = [r.id for r in model.reactions if '[d]' in r.id and r.id.startswith('EX_')]
    for rxn_id in diet_rxns:
        new_id = rxn_id.replace('EX_', 'Diet_EX_')
        if new_id not in model.reactions:
            model.reactions.get_by_id(rxn_id).id = new_id

    # First: Set ALL Diet_EX_ reactions to lower bound 0 (like useDiet does)
    for rxn in model.reactions:
        if rxn.id.startswith('Diet_EX_'):
            rxn.lower_bound = 0

    # Apply diet
    for _, row in diet_constraints.iterrows():
        rxn = row['rxn_id']
        if rxn in model.reactions:
            model.reactions.get_by_id(rxn).lower_bound = float(row['lower_bound'])
            if pd.notnull(row['upper_bound']):
                model.reactions.get_by_id(rxn).upper_bound = float(row['upper_bound'])

    print(f"Processing {samp}: diet applied")
    
    # Constrain community biomass
    if 'communityBiomass' in model.reactions:
        model.reactions.communityBiomass.lower_bound = lower_bm
        model.reactions.communityBiomass.upper_bound = upper_bm

    for rxn in model.reactions:
        if rxn.id.startswith('UFEt_') or rxn.id.startswith('DUt_') or rxn.id.startswith('EX_'):
            rxn.upper_bound = 1e6

    # Change the bound of the humanMets if not included in the diet
    # BUT it is in the existing model's reactions
    for met_id, bound in humanMets.items():
        rxn_id = f'Diet_EX_{met_id}[d]'
        if rxn_id not in diet_constraints['rxn_id'].values and rxn_id in model.reactions:
            model.reactions.get_by_id(rxn_id).bounds = bound, 10000.


    # close demand and limit sink reactions
    for rxn in model.reactions:
        if '_DM_' in rxn.id:
            rxn.lower_bound = 0
        elif '_sink_' in rxn.id:
            rxn.lower_bound = -1 

    # Objective: EX_microbeBiomass[fe]
    model.objective = 'EX_microbeBiomass[fe]'
    model.optimize()

In [ ]:
save_dir = os.path.join('Python_Models2', 'Diet')
os.makedirs(save_dir, exist_ok=True)

model_dict = make_mg_pipe_model_dict(
    model, C=model_data['C'], d=model_data['d'], dsense=model_data['dsense'], ctrs=model_data['ctrs']
)

diet_model_path = os.path.join(save_dir, f"microbiota_model_diet_{samp}.mat")
savemat(diet_model_path, {'model': model_dict}, oned_as='column')

print(f"Processing {samp}: starting fva")

fecal_rxn_ids = [model.reactions.index(rxn) for rxn in model.exchanges]

diet_rxn_ids = [rxn.id.replace('EX_', 'Diet_EX_').replace('[fe]', '[d]') for rxn in model.exchanges]
diet_rxn_ids = [model.reactions.index(model.reactions.get_by_id(rid)) for rid in diet_rxn_ids if rid in model.reactions]

A, rhs, csense, lb, ub, c = build_constraint_matrix(diet_model_path)
opt_model, vars, obj_expr = build_optlang_model(A, rhs, csense, lb, ub, c)

In [ ]:
opt_model.objective = Objective(obj_expr, direction='max')
print(f'Model Status after optimization: {opt_model.optimize()}')

In [ ]:
min_flux_fecal, max_flux_fecal = run_sequential_fva(opt_model, vars, obj_expr, fecal_rxn_ids, opt_percentage=99.99)
min_flux_diet, max_flux_diet = run_sequential_fva(opt_model, vars, obj_expr, diet_rxn_ids, opt_percentage=99.99)

net_production[samp] = {}
net_uptake[samp] = {}


# exchanges derived from exMets (all exchanged metabolites across all individual models) -> intersect it with rxns in this particular model
fecal_rxns = [r.id for r in model.exchanges]
diet_rxns = [rxn.replace('EX_', 'Diet_EX_').replace('[fe]', '[d]') for rxn in fecal_rxns]
exchanges = set(fecal_rxns).intersection(set(exchanges))

# cut off very small values below solver sensitivity
tol = 1e-07
fecal_var_map = dict(zip(fecal_rxns, max_flux_fecal))
diet_var_map = dict(zip(diet_rxns, max_flux_diet))
for rxn in fecal_rxns:
    fecal = rxn
    diet = rxn.replace('EX_', 'Diet_EX_').replace('[fe]', '[d]')
    fecal_var = fecal_var_map.get(fecal, None)
    diet_var = diet_var_map.get(diet, None)

    if abs(max_flux_fecal.get(fecal_var, 0)) < tol: max_flux_fecal.get(fecal_var, 0) == 0

    prod = abs(min_flux_diet.get(diet_var, 0) + max_flux_fecal.get(fecal_var, 0))
    uptk = abs(max_flux_diet.get(diet_var, 0) + min_flux_fecal.get(fecal_var, 0))
    net_production[samp][rxn] = prod
    net_uptake[samp][rxn] = uptk

exchanges = sorted(exchanges)

prod_data = {samp: [net_production[samp].get(r, 0) for r in exchanges] for samp in samp_names}
net_secretion_df = pd.DataFrame(prod_data, index=exchanges)

net_secretion_df

## Use personalized matlab to get diet using our codes

In [ ]:
clean_samp_names, organisms, ex_mets = get_individual_size_name(
    abun_file_path='test_data_input/normCoverageReduced.csv',
    mod_path='test_data_input/AGORA103'
)

personalized_matmodel = cobra.io.load_matlab_model('Matlab_Models\Personalized\microbiota_model_samp_SRS011061.mat')

In [ ]:
samp_names = ['SRS011061']
model_dir = 'test_data_input/AGORA103'
diet_file = 'test_data_input\AverageEU_diet_fluxes.txt'
res_path = 'zzzresults/Matlab'
lower_bm=0.4 
upper_bm=1.0
solver='cplex'

os.makedirs(res_path, exist_ok=True)
exchanges = [f"EX_{m.replace('[e]', '[fe]')}" for m in ex_mets if m != 'biomass[e]']

net_production = {}
net_uptake = {}

# define human-derived metabolites present in the gut: primary bile acids, amines, mucins, host glycans
humanMets = {
    'gchola': -10, 'tdchola': -10, 'tchola': -10, 'dgchol': -10,
    '34dhphe': -10, '5htrp': -10, 'Lkynr': -10, 'f1a': -1,
    'gncore1': -1, 'gncore2': -1, 'dsT_antigen': -1, 'sTn_antigen': -1,
    'core8': -1, 'core7': -1, 'core5': -1, 'core4': -1,
    'ha': -1, 'cspg_a': -1, 'cspg_b': -1, 'cspg_c': -1,
    'cspg_d': -1, 'cspg_e': -1, 'hspg': -1
}

# Adapt diet
diet_constraints = adapt_vmh_diet_to_agora(diet_file, setup_used='Microbiota')

In [ ]:
for samp in tqdm(samp_names):
    model = personalized_matmodel
    model_data = loadmat('Matlab_Models\Personalized\microbiota_model_samp_SRS011061.mat', simplify_cells=True)['model']
    model.solver = solver
    model.name = samp

    print(f"Processing {samp}: got model")

    # Before applying diet constraints, update reaction IDs in the model
    diet_rxns = [r.id for r in model.reactions if '[d]' in r.id and r.id.startswith('EX_')]
    for rxn_id in diet_rxns:
        new_id = rxn_id.replace('EX_', 'Diet_EX_')
        if new_id not in model.reactions:
            model.reactions.get_by_id(rxn_id).id = new_id

    # First: Set ALL Diet_EX_ reactions to lower bound 0 (like useDiet does)
    for rxn in model.reactions:
        if rxn.id.startswith('Diet_EX_'):
            rxn.lower_bound = 0

    # Apply diet
    for _, row in diet_constraints.iterrows():
        rxn = row['rxn_id']
        if rxn in model.reactions:
            model.reactions.get_by_id(rxn).lower_bound = float(row['lower_bound'])
            if pd.notnull(row['upper_bound']):
                model.reactions.get_by_id(rxn).upper_bound = float(row['upper_bound'])

    print(f"Processing {samp}: diet applied")
    
    # Constrain community biomass
    if 'communityBiomass' in model.reactions:
        model.reactions.communityBiomass.lower_bound = lower_bm
        model.reactions.communityBiomass.upper_bound = upper_bm

    for rxn in model.reactions:
        if rxn.id.startswith('UFEt_') or rxn.id.startswith('DUt_') or rxn.id.startswith('EX_'):
            rxn.upper_bound = 1e6

    # Change the bound of the humanMets if not included in the diet
    # BUT it is in the existing model's reactions
    for met_id, bound in humanMets.items():
        rxn_id = f'Diet_EX_{met_id}[d]'
        if rxn_id not in diet_constraints['rxn_id'].values and rxn_id in model.reactions:
            model.reactions.get_by_id(rxn_id).bounds = bound, 10000.


    # close demand and limit sink reactions
    for rxn in model.reactions:
        if '_DM_' in rxn.id:
            rxn.lower_bound = 0
        elif '_sink_' in rxn.id:
            rxn.lower_bound = -1 

    # Objective: EX_microbeBiomass[fe]
    model.objective = 'EX_microbeBiomass[fe]'
    model.optimize()

In [ ]:
save_dir = os.path.join(res_path, 'Diet')
os.makedirs(save_dir, exist_ok=True)

model_dict = make_mg_pipe_model_dict(
    model, C=model_data['C'], d=model_data['d'], dsense=model_data['dsense'], ctrs=model_data['ctrs']
)

diet_model_path = os.path.join(save_dir, f"microbiota_model_diet_{samp}.mat")
savemat(diet_model_path, {'model': model_dict}, oned_as='column')

print(f"Processing {samp}: starting fva")

In [ ]:
fecal_rxn_ids = [model.reactions.index(rxn) for rxn in model.exchanges]

diet_rxn_ids = [rxn.id.replace('EX_', 'Diet_EX_').replace('[fe]', '[d]') for rxn in model.exchanges]
diet_rxn_ids = [model.reactions.index(model.reactions.get_by_id(rid)) for rid in diet_rxn_ids if rid in model.reactions]

A, rhs, csense, lb, ub, c = build_constraint_matrix(diet_model_path)
opt_model, vars, obj_expr = build_optlang_model(A, rhs, csense, lb, ub, c)

In [ ]:
min_flux_fecal, max_flux_fecal = run_sequential_fva(opt_model, vars, obj_expr, fecal_rxn_ids, opt_percentage=99.99)
min_flux_diet, max_flux_diet = run_sequential_fva(opt_model, vars, obj_expr, diet_rxn_ids, opt_percentage=99.99)

net_production[samp] = {}
net_uptake[samp] = {}


# exchanges derived from exMets (all exchanged metabolites across all individual models) -> intersect it with rxns in this particular model
fecal_rxns = [r.id for r in model.exchanges]
diet_rxns = [rxn.replace('EX_', 'Diet_EX_').replace('[fe]', '[d]') for rxn in fecal_rxns]
exchanges = set(fecal_rxns).intersection(set(exchanges))

# cut off very small values below solver sensitivity
tol = 1e-07
fecal_var_map = dict(zip(fecal_rxns, max_flux_fecal))
diet_var_map = dict(zip(diet_rxns, max_flux_diet))
for rxn in fecal_rxns:
    fecal = rxn
    diet = rxn.replace('EX_', 'Diet_EX_').replace('[fe]', '[d]')
    fecal_var = fecal_var_map.get(fecal, None)
    diet_var = diet_var_map.get(diet, None)

    if abs(max_flux_fecal.get(fecal_var, 0)) < tol: max_flux_fecal.get(fecal_var, 0) == 0

    prod = abs(min_flux_diet.get(diet_var, 0) + max_flux_fecal.get(fecal_var, 0))
    uptk = abs(max_flux_diet.get(diet_var, 0) + min_flux_fecal.get(fecal_var, 0))
    net_production[samp][rxn] = prod
    net_uptake[samp][rxn] = uptk

In [ ]:
exchanges = sorted(exchanges)

prod_data = {samp: [net_production[samp].get(r, 0) for r in exchanges] for samp in samp_names}
net_secretion_df = pd.DataFrame(prod_data, index=exchanges)

net_secretion_df